In [5]:
import requests
import json
from pprint import pprint
from RiotRequest import RiotRequest

api_key = "RGAPI-889ae58e-f777-433a-ab22-dbcb3cda10d2"

def get_account_id(summoner_name):
    '''
    input: 접근할 수 있는 summoner name
    output: account_id
    '''
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{0}?api_key={1}".format(summoner_name, api_key))
    return json_data['accountId']


def get_match_ids(account_id):
    '''
    matches라는 key 안에 최대 최근 100개의 match id만
    API doc: https://developer.riotgames.com/api-methods/#match-v4/GET_getMatchlist
    '''
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{0}?api_key={1}".format(account_id, api_key))
    game_id_list = [match['gameId'] for match in json_data['matches']]
    return game_id_list

def get_history(account_id):
    beginIndex = 0
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{0}?api_key={1}&beginIndex={2}".format(account_id, api_key, beginIndex))
    
    return json_data
    
def get_match_history(account_id):
    '''
    account_id: str. accound id of the user
    output: List[MatchReferenceDto]
    MatchReferenceDto: {'platformId', 'gameId', 'champion', 'queue', 'season', 'timestamp', 'role', 'lane'}
    API doc: https://developer.riotgames.com/api-methods/#match-v4/GET_getMatchlist
    '''   
    # initialize
    beginIndex = 0
    matchList = []
    
    # make multiple requests to get all matches
    while True: 
        url = "https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{0}?api_key={1}&beginIndex={2}".format(account_id, api_key, beginIndex)
        json_data = RiotRequest(url)
        
        # save match data into list
        matchList.extend([match['gameId'] for match in json_data['matches']])

        if json_data['endIndex'] >= json_data['totalGames']: # if endIndex reaches the number of total games
            break # terminate the while loop
        else: # make a request on previous 100 matches
            beginIndex += 100
    
    return matchList

def get_match_history_by_season(account_id, season):
    # beginTime/endTime of each season: https://rankedboost.com/season-end-league-of-legends/
    pass


def get_game_detail(game_id):
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/match/v4/matches/{0}?api_key={1}".format(game_id, api_key))
    return json_data

def get_match_timeline(game_id):
    '''
    1분 단위로 나뉘어진 프레임의 리스트를 반환
    List[{'timestamp': int, 'participantFrames': Map[String, MatchParticipantFrameDto], 'events': List[MatchEventDto]}]
    '''
    json_data = RiotRequest("https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/{0}?api_key={1}".format(game_id, api_key))
    return json_data['frames']

티어 별 SUMMONERID 구하기

In [7]:
def get_tiers(queue, tier, division):
    queue = 'RANKED_SOLO_5x5'
    page = 1
    summonerId= []
    
    # make multiple requests to get all ID
    while True: 
        json_data = RiotRequest("https://kr.api.riotgames.com/lol/league/v4/entries/{0}/{1}/{2}?api_key={3}&page={4}".format(queue, tier, division, api_key, page))
        
        # save summonerNamesinto list
        summonerId.extend([usr['summonerId'] for usr in json_data])

        if len(json_data) < 205: # 페이지에 있는 유저의 수가 205보다 작으면
            break # 끝냄
        else: # 205일 경우에는 page를 더해준다
            page += 1
    return summonerId

In [3]:
DIAMOND1 = get_tiers('RANKED_SOLO_5x5', 'DIAMOND', 'I')

In [4]:
DIAMOND2 = get_tiers('RANKED_SOLO_5x5', 'DIAMOND', 'II')

In [6]:
DIAMOND3 = get_tiers('RANKED_SOLO_5x5', 'DIAMOND', 'III')

In [7]:
DIAMOND4 = get_tiers('RANKED_SOLO_5x5', 'DIAMOND', 'IV')

Exceeded Rate Limiting. Will retry After 65 seconds
Exceeded Rate Limiting. Will retry After 64 seconds
Exceeded Rate Limiting. Will retry After 57 seconds
Exceeded Rate Limiting. Will retry After 59 seconds


In [8]:
PLATINUM1 = get_tiers('RANKED_SOLO_5x5', 'PLATINUM', 'I')

Exceeded Rate Limiting. Will retry After 62 seconds


In [9]:
PLATINUM2 = get_tiers('RANKED_SOLO_5x5', 'PLATINUM', 'II')

Exceeded Rate Limiting. Will retry After 62 seconds
Exceeded Rate Limiting. Will retry After 59 seconds
Exceeded Rate Limiting. Will retry After 60 seconds


In [10]:
PLATINUM3 = get_tiers('RANKED_SOLO_5x5', 'PLATINUM', 'III')

Exceeded Rate Limiting. Will retry After 60 seconds
Exceeded Rate Limiting. Will retry After 58 seconds
Exceeded Rate Limiting. Will retry After 65 seconds


In [11]:
PLATINUM4 = get_tiers('RANKED_SOLO_5x5', 'PLATINUM', 'IV')

Exceeded Rate Limiting. Will retry After 66 seconds
Exceeded Rate Limiting. Will retry After 54 seconds
Exceeded Rate Limiting. Will retry After 61 seconds
Exceeded Rate Limiting. Will retry After 59 seconds
Exceeded Rate Limiting. Will retry After 57 seconds
Exceeded Rate Limiting. Will retry After 61 seconds
Exceeded Rate Limiting. Will retry After 62 seconds
Exceeded Rate Limiting. Will retry After 60 seconds


In [6]:
#name of master
queue = 'RANKED_SOLO_5x5'
json_data = RiotRequest("https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/{0}?api_key={1}".format(queue, api_key))
MASTER = [i['summonerId'] for i in json_data['entries']]

In [21]:
#name of grandmaster
json_data = RiotRequest("https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/{0}?api_key={1}".format(queue, api_key))
GRANDMASTER = [i['summonerId'] for i in json_data['entries']]


In [7]:
#name of CHALLENGER
queue = 'RANKED_SOLO_5x5'
json_data = RiotRequest("https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{0}?api_key={1}".format(queue, api_key))
CHALLENGER = [i['summonerId'] for i in json_data['entries']]


각 티어별 인원수 파악

In [11]:
len(CHALLENGER)

300

In [21]:
len(GRANDMASTER)

700

In [22]:
len(MASTER)

1202

In [23]:
len(DIAMOND1)

4246

In [24]:
len(DIAMOND2)

8005

In [25]:
len(DIAMOND3)

17898

In [26]:
len(DIAMOND4)

65534

In [16]:
len(PLATINUM1)

38642

In [15]:
len(PLATINUM2)

50666

In [14]:
len(PLATINUM3)

66071

In [13]:
len(PLATINUM4)

155117

#ACCOUNT_ID 구하기

In [8]:
def account_id(summonerId):
    '''
    input: 접근할 수 있는 summoner Id
    output: account_id
    '''
    try :
        json_data = RiotRequest("https://kr.api.riotgames.com/lol/summoner/v4/summoners/{0}?api_key={1}".format(summonerId, api_key))
    except: TypeError
    
    return json_data['accountId']

In [9]:
#ACCOUNT_ID 구하기
account_CH = []
for i in CHALLENGER:
    account_CH.append(account_id(i))

Exceeded Rate Limiting. Will retry After 51 seconds
Exceeded Rate Limiting. Will retry After 81 seconds
Exceeded Rate Limiting. Will retry After 80 seconds


In [41]:
for i in GRANDMASTER:
    account_GR = account_id(i)

Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 97 seconds


In [43]:
for i in MASTER:
    account_MA = account_id(i)

Exceeded Rate Limiting. Will retry After 27 seconds
Exceeded Rate Limiting. Will retry After 98 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 96 seconds


In [44]:
for i in DIAMOND1:
    account_D1 = account_id(i)

Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 96 seconds
Exceeded Rate Limiting. Will retry After 98 seconds
Exceeded Rate Limiting. Will retry After 98 seconds
Exceeded Rate Limiting. Will retry After 97 seconds
Exceeded Rate Limiting. Will retry After 98 seconds
Exceeded Rate Limiting. Will retry After 98 seconds
Exceeded Rate Limiting. Will retry After 98 seconds
Exceeded Rat

In [50]:
for i in DIAMOND2:
    account_D2 = account_id(i)

Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 91 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 91 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rat

TypeError: 'NoneType' object is not subscriptable

In [51]:
for i in DIAMOND3:
    account_D3 = account_id(i)

Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 90 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rat

Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 93 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rate Limiting. Will retry After 71 seconds
Exceeded Rate Limiting. Will retry After 92 seconds
Exceeded Rat

In [ ]:
for i in DIAMOND4:
    account_D4 = account_id(i)

In [ ]:
for i in PLATINUM1:
    account_P1 = account_id(i)

In [ ]:
for i in PLATINUM2:
    account_P2 = account_id(i)

In [ ]:
for i in PLATINUM3:
    account_P3 = account_id(i)

In [ ]:
for i in PLATINUM4:
    account_P4 = account_id(i)

In [15]:
import pickle

In [37]:
with open("account_CH.pickle", 'wb') as fw:
    pickle.dump(account_CH, fw)
with open("match_CH.pickle", "wb") as fw:
    pickle.dump(match_CH, fw)
with open("match_CH_dict.pickle", "wb") as fw:
    pickle.dump(match_CH_dict, fw) 

In [7]:
len(accountlist_D4)

65956

In [9]:
with open('GM_LANE.pickle', 'rb') as f:
    GM_LANE=pickle.load(f)

In [10]:
GM_LANE

[{'U1Iu4V_UmTVG-W-2BUt4bPh5yYqa5kkBUFCVK8D3bQNm': {'TOP': 6.9,
   'MID': 42.6,
   'JUNGLE': 3.2,
   'DUO_CARRY': 0.9,
   'DUO_SUPPORT': 2.9,
   'NONE': 43.4}},
 {'OKeVZsy-IvumdKgYQ0C5JK_0t2E1YH4Tzk-3F6Vy3OIKu4Q': {'TOP': 8.9,
   'MID': 54.0,
   'JUNGLE': 6.3,
   'DUO_CARRY': 2.2,
   'DUO_SUPPORT': 1.0,
   'NONE': 27.7}},
 {'5rfgdda5BTlNENHq-lIp4FUC94JNtnUpM0z4kLgcexYqVmQ': {'TOP': 7.2,
   'MID': 47.6,
   'JUNGLE': 1.9,
   'DUO_CARRY': 3.1,
   'DUO_SUPPORT': 0.6,
   'NONE': 39.6}},
 {'-slKvGg5PLQ2YT8VIs8RdVaq57jTpDjkTQOh1WhwYmDCWg8': {'TOP': 10.0,
   'MID': 60.8,
   'JUNGLE': 1.1,
   'DUO_CARRY': 0.3,
   'DUO_SUPPORT': 1.7,
   'NONE': 26.0}},
 {'ZszNs51p2rex6HwPKlIzyl0j7qHPjyLzpHJlRcyszeA0qKhE1fKPg22G': {'TOP': 1.3,
   'MID': 4.0,
   'JUNGLE': 1.3,
   'DUO_CARRY': 0.3,
   'DUO_SUPPORT': 67.9,
   'NONE': 25.3}},
 {'N7yUefnvWBEjO-4DMyGV8tC21XIi7GGLBwxKXRbZP1Fwqg4': {'TOP': 3.0,
   'MID': 9.9,
   'JUNGLE': 2.5,
   'DUO_CARRY': 45.2,
   'DUO_SUPPORT': 5.8,
   'NONE': 33.6}},
 {'XnAe-u49SF8b

In [16]:
GM_LANE[1]['OKeVZsy-IvumdKgYQ0C5JK_0t2E1YH4Tzk-3F6Vy3OIKu4Q']

{'TOP': 8.9,
 'MID': 54.0,
 'JUNGLE': 6.3,
 'DUO_CARRY': 2.2,
 'DUO_SUPPORT': 1.0,
 'NONE': 27.7}

In [12]:
import operator

In [21]:
sdict = sorted(GM_LANE[1]['OKeVZsy-IvumdKgYQ0C5JK_0t2E1YH4Tzk-3F6Vy3OIKu4Q'].items(), key = operator.itemgetter(1), reverse = True)

In [22]:
sdict

[('MID', 54.0),
 ('NONE', 27.7),
 ('TOP', 8.9),
 ('JUNGLE', 6.3),
 ('DUO_CARRY', 2.2),
 ('DUO_SUPPORT', 1.0)]

In [25]:
newdict = {}
for i in range(0,6):
    newdict[sdict[i][0]]=sdict[i][1]
print(newdict)

{'MID': 54.0, 'NONE': 27.7, 'TOP': 8.9, 'JUNGLE': 6.3, 'DUO_CARRY': 2.2, 'DUO_SUPPORT': 1.0}


In [15]:
account_CH

['IOQjr22mIGRl4QPIdsPCAmlkqL00t3bZR7vLHHuABQ0Gqh4',
 'mDcnV0ZA1hyRB6xtFOpXA_q_WEaR2bc3Y72PTE1rAvShb1Ofg-MiUP5S',
 'B494lJrFQkxnp73DFuIMv0V2RiIvjW0rN8Z0bJS1WMY19U8',
 'c6n6K9Bl8pumqu2IO6WEUkiwutXvHNi0MBGe64B3JI5dbiI',
 'PA8keq17Il74Ywi58ft1IijSOted9oDEbfMWZkgazh5JKUhuJh3Klkfx',
 'ez_Kh_gSmo_uUY6mCfYC6IpiLV0UDYZ528xvgtvXKjqRebH21Dz3zD3F',
 'USt0VWvzNit9DKo2QHgOpJOeqr1Gzi2cOAVI0QzmcX5oYRUo8h-h_R4n',
 '3KIPOa36G6BBHn6q4MmewP6pyXQyuO0sE2VzQGM2z70ofUs',
 'SCMsiCvAL28BBN9yIBzL-gE_6P1QjsXh3yW8P_ru5Bz1uaEcp1a1W14y',
 '7t49L7KrVczc1uUK562hHDCgZjxXqeNhpGgqYF6f33Edi54',
 'TY4-Syj9PoV1DzxMXsxHeuY2pm3aTpS-uyT7NJdoXlMp1XA',
 '0oUhdI5Z0E1psWdSU2RA9DyCKACh2P7UhfuNWmE1RuhLWGU',
 'WqhPqFx4TnFemBa71OpWyb8pKQe-n7aX6KAJkhq9Y9yUy48',
 'bTX6kpKTeq5rUIWZSd9N_XF21NN2whO0O096Kox0yuRIO3QqpOufQRg5',
 'gA-_lp9fLS5vZhxrbtFeIHT9CY_3pzHtH2JHk8QdS8b91g0',
 'xV_-tgAJdLWloeh0JQaGc9FGB9KyJCyOKkLisluENvF0t8Y',
 '0SvlzSmrDkPNep3r4Oe7kTkpVv_nog5vRz4hdtuYYM2-G1o',
 'h6st_nS5WxvZdCsgMiYdVWm_eYL0YJk7ugAV7jrZK1YXbWzwpncuOTkt',
 

In [10]:
match_CH = []
for i in account_CH:
    match_CH.append(get_match_ids(i))

Exceeded Rate Limiting. Will retry After 45 seconds
Exceeded Rate Limiting. Will retry After 62 seconds
Exceeded Rate Limiting. Will retry After 56 seconds


In [28]:
len(match_CH)

300

In [26]:
match_CH

[[3719452465,
  3719423940,
  3719314377,
  3719230864,
  3719058655,
  3719065186,
  3718988559,
  3719001564,
  3718917735,
  3718900976,
  3718749347,
  3718225864,
  3718116803,
  3718099814,
  3718069513,
  3717448643,
  3717271003,
  3716362409,
  3715116880,
  3715066619,
  3715038793,
  3714995906,
  3715000054,
  3714602777,
  3714555313,
  3714477894,
  3713635479,
  3713578938,
  3713593693,
  3713527563,
  3713504742,
  3713530859,
  3713209819,
  3713210642,
  3713114235,
  3712640400,
  3712082664,
  3712035941,
  3711454434,
  3710930926,
  3710637848,
  3710353657,
  3710291652,
  3710005587,
  3709904870,
  3709653521,
  3709432879,
  3709277494,
  3709214962,
  3709180308,
  3709117232,
  3709151972,
  3709064029,
  3709047522,
  3709042829,
  3708939171,
  3708936774,
  3708934141,
  3708980439,
  3707441260,
  3706111511,
  3706042545,
  3705961001,
  3705915824,
  3705460394,
  3705325596,
  3704886969,
  3704350874,
  3703816177,
  3703822733,
  3703759811,
  3703

In [30]:
match_CH_dict = {}
i=0
for id in account_CH:
    match_CH_dict[id]= match_CH[i]
    i+=1

In [31]:
pprint(match_CH_dict)

{'-HiJOyutqWavwOdwJuQQagq_2rJ7uyT1LpnGphniO_zF': [3719362544,
                                                  3719311229,
                                                  3719302963,
                                                  3716234033,
                                                  3712020241,
                                                  3710291652,
                                                  3710209106,
                                                  3710233489,
                                                  3710185509,
                                                  3709289723,
                                                  3708732097,
                                                  3708685801,
                                                  3708609542,
                                                  3708612606,
                                                  3707216816,
                                                  3707222994,
        

                                                     3663201766,
                                                     3663158329,
                                                     3663125425,
                                                     3663091638,
                                                     3662885739,
                                                     3662840867,
                                                     3661569359,
                                                     3661535973,
                                                     3661532348,
                                                     3661415485,
                                                     3661307895,
                                                     3661331199,
                                                     3661235746,
                                                     3661198217,
                                                     3661171123,
                         

                                                     3717500351,
                                                     3714378376,
                                                     3714340016,
                                                     3714169465,
                                                     3714043031,
                                                     3713835713,
                                                     3713701022,
                                                     3713060942,
                                                     3712929971,
                                                     3712933775,
                                                     3709653521,
                                                     3709291473,
                                                     3709225352,
                                                     3708956239,
                                                     3708952339,
                         

                                                     3698408919,
                                                     3695308339,
                                                     3694665970,
                                                     3694608382,
                                                     3694456608,
                                                     3694412615,
                                                     3694389548,
                                                     3694354313,
                                                     3694336403],
 'B494lJrFQkxnp73DFuIMv0V2RiIvjW0rN8Z0bJS1WMY19U8': [3719539602,
                                                     3719487971,
                                                     3719298175,
                                                     3719265167,
                                                     3719048869,
                                                     3718970255,
                        

                                                     3714311207,
                                                     3714193908,
                                                     3714049131,
                                                     3713024040,
                                                     3713003244,
                                                     3712898038,
                                                     3712910103,
                                                     3712763652,
                                                     3712116212,
                                                     3712120906,
                                                     3712083241,
                                                     3711703108,
                                                     3711687700,
                                                     3711662201,
                                                     3711465086,
                         

...

                                                     3716005922,
                                                     3715949117,
                                                     3715869473,
                                                     3715799379,
                                                     3715773307,
                                                     3715677272,
                                                     3715598983,
                                                     3715575231,
                                                     3715109345,
                                                     3715055161,
                                                     3714999656,
                                                     3714993909,
                                                     3714958671,
                                                     3714963767,
                                                     3714897776,
                         

                                                     3700472480,
                                                     3700454552,
                                                     3700423402,
                                                     3700368768,
                                                     3700375036,
                                                     3700349621,
                                                     3700334391,
                                                     3700287592,
                                                     3700218146,
                                                     3700145219,
                                                     3699876671,
                                                     3699921879,
                                                     3699779143,
                                                     3699696362,
                                                     3699525785,
                         

                                                  3718014045,
                                                  3718001579,
                                                  3717978683,
                                                  3717948244,
                                                  3717727986,
                                                  3717499423,
                                                  3717297454,
                                                  3716288205,
                                                  3716217468,
                                                  3714652412,
                                                  3714612594,
                                                  3713657365,
                                                  3713642620,
                                                  3713576171,
                                                  3713551890,
                                                  3713537225,
        

                                                              3703773956,
                                                              3703727724,
                                                              3703733916],
 'u9rwa90SvrxIJdaLZVg3T8XKyRv4_pElQucj0EaAg90J': [3719048230,
                                                  3719036171,
                                                  3719021027,
                                                  3718994342,
                                                  3718929787,
                                                  3718040419,
                                                  3718000286,
                                                  3709907667,
                                                  3709893878,
                                                  3709750534,
                                                  3709602312,
                                                  3709462806,
                                 

                                                  3691252663,
                                                  3690077296,
                                                  3690066327,
                                                  3690084501,
                                                  3690034474,
                                                  3689989673,
                                                  3689790700,
                                                  3689675301,
                                                  3689628608,
                                                  3689625367,
                                                  3688940329,
                                                  3688488254,
                                                  3688444228,
                                                  3688415907],
 'wkYDXJL0m508eYJlGnj18O5GOvoLMBCL0ZIjrxx8JXTq__M': [3719460906,
                                                     3719402424,
 

In [16]:
pprint(get_game_detail(3719269150))

{'gameCreation': 1561444315589,
 'gameDuration': 1234,
 'gameId': 3719269150,
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'gameVersion': '9.12.277.6038',
 'mapId': 11,
 'participantIdentities': [{'participantId': 1,
                            'player': {'accountId': '0m4oYoRs3EYUY1TKgiHH3QFckOjoJFVwH4Yr--dU8uLxjUpEyGF8_WWB',
                                       'currentAccountId': '0m4oYoRs3EYUY1TKgiHH3QFckOjoJFVwH4Yr--dU8uLxjUpEyGF8_WWB',
                                       'currentPlatformId': 'KR',
                                       'matchHistoryUri': '/v1/stats/player_history/KR/2062694685642208',
                                       'platformId': 'KR',
                                       'profileIcon': 4099,
                                       'summonerId': 'g4yvMRscG-Z8H4R_xk2dudLI7NXbqIweMwh2BS8OL0DgKSU',
                                       'summonerName': 'mouheimangdeiyan'}},
                           {'participantId': 2,
                       

                             'trueDamageDealt': 1244,
                             'trueDamageDealtToChampions': 356,
                             'trueDamageTaken': 441,
                             'turretKills': 3,
                             'unrealKills': 0,
                             'visionScore': 14,
                             'visionWardsBoughtInGame': 0,
                             'wardsKilled': 4,
                             'wardsPlaced': 6,
                             'win': True},
                   'teamId': 200,
                   'timeline': {'creepsPerMinDeltas': {'0-10': 8, '10-20': 11},
                                'csDiffPerMinDeltas': {'0-10': 0.050000000000000266,
                                                       '10-20': 2},
                                'damageTakenDiffPerMinDeltas': {'0-10': -64.20000000000002,
                                                                '10-20': -256.65000000000003},
                                'dama

In [36]:
win_rate_all = {}

champlist = {}
for account in match_CH_dict: #match_CH에 저장된 account_id를 돌림
    win = 0  # 100경기를 돌리는데, 이기는 경기를 1로 저장해 모두 더하자.
    for game in range(0,100): #총 100경기씩 저장되어있으므로, 한 account_id에 100경기 씩 돌려야한다.
       
        for i in range(0,10):  #한경기에 10명의 플레이어가 있음         
            if get_game_detail(match_CH_dict[account][game])[ 'participantIdentities'][i]['player']['accountId'] ==  account: #10명의 플레이어중 지금 우리가 조사하는 사람의 승을 알고 싶기 때문에, 원래의 키값과 같은 사람을 골라낸다.
                that_number = i # 키값이 일치하는 사람의 순서
                
                if get_game_detail(match_CH_dict[account][game])['participants'][that_number]['stats']['win'] == True: #만약 그 사람이 이겼는지 졌는지 를 확인하고
                    win+=1
                    
    print(account)
    print(win)
    win_rate_all[account] = win

print(win_rate_all)

From cffi callback <function _verify_callback at 0x000002318DD20730>:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='kr.api.riotgames.com', port=443): Max retries exceeded with url: /lol/match/v4/matches/3719065186?api_key=RGAPI-889ae58e-f777-433a-ab22-dbcb3cda10d2 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [64]:
for a in match_CH:
    for b in range(0,100):
        a[b]
print(a[b])

3702047685


In [24]:
for account in match_CH: #match_CH에 저장된 account_id를 돌림
    for game in range(0,100): #총 100경기씩 저장되어있으므로, 한 account_id에 100경기 씩 돌려야한다.
        win_list = [] # 100경기를 돌리는데, 이기는 경기를 1로 저장해 모두 더하자.
        for i in range(0,10):  #한경기에 10명의 플레이어가 있음         
            if get_game_detail(account[game])[ 'participantIdentities'][i]['player']['accountId'] ==  account_CH[game]: #10명의 플레이어중 지금 우리가 조사하는 사람의 승을 알고 싶기 때문에, 원래의 키값과 같은 사람을 골라낸다.
                that_number = i # 키값이 일치하는 사람의 순서
                print(that_number)
            else:
                pass

2
Exceeded Rate Limiting. Will retry After 60 seconds
Exceeded Rate Limiting. Will retry After 61 seconds
Exceeded Rate Limiting. Will retry After 57 seconds
8
Exceeded Rate Limiting. Will retry After 61 seconds
Exceeded Rate Limiting. Will retry After 61 seconds
Exceeded Rate Limiting. Will retry After 62 seconds


KeyboardInterrupt: 